In [1]:
import tensorflow as tf
import numpy as np
from rnn import ImageCaptioning
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from cs231n.image_utils import image_from_url

%load_ext autoreload
%autoreload 2

In [2]:
data = load_coco_data(pca_features=True)

# Print out all the keys and values from the data dictionary
for k, v in data.iteritems():
  if type(v) == np.ndarray:
    print k, type(v), v.shape, v.dtype
  else:
    print k, type(v), len(v)

idx_to_word <type 'list'> 1004
train_captions <type 'numpy.ndarray'> (400135, 17) int32
val_captions <type 'numpy.ndarray'> (195954, 17) int32
train_image_idxs <type 'numpy.ndarray'> (400135,) int32
val_features <type 'numpy.ndarray'> (40504, 512) float32
val_image_idxs <type 'numpy.ndarray'> (195954,) int32
train_features <type 'numpy.ndarray'> (82783, 512) float32
train_urls <type 'numpy.ndarray'> (82783,) |S63
val_urls <type 'numpy.ndarray'> (40504,) |S63
word_to_idx <type 'dict'> 1004


In [3]:
word_to_idx = data['word_to_idx']
features = data['train_features'][0:100]
captions = data['train_captions'][0:100]
train_urls = data['train_urls'][0:100]

In [6]:
image_captioning = ImageCaptioning(word_to_idx, batch_size= 100, dim_feature=512, dim_embed=128,
                                   dim_hidden=128, n_time_step=16, dtype=tf.float32)

logits, loss = image_captioning.build_model(features, captions)

In [7]:
optimizer = tf.train.AdamOptimizer(0.03).minimize(loss)

In [16]:
idx_to_word = {i: w for w, i in word_to_idx.iteritems()}

In [17]:
start_char = idx_to_word[captions[0,0]]

In [20]:
start_char

u'<START>'

In [26]:
n_epochs = 200
with tf.Session() as sess:
    tf.initialize_all_variables().run()

    for e in range(n_epochs):
        _, l = sess.run([optimizer, loss])
        print "loss at epoch %d: %f" %(e, l)  
        if e % 10 == 0:
            score = sess.run(logits)  # (N, T, V)
            idxs = np.argmax(score, 2)  # (N, T)
            idx = idxs[0]  # (T)
            sentence = [idx_to_word[i]+" " for i in idx]
            print "generated sentence: %s" %(start_char+''.join(sentence))

loss at epoch 0: 7580.907227
generated sentence: <START>talking very restroom <END> stainless decorated table well <END> topped vanity cellphone at sinks for streets 
loss at epoch 1: 6508.325195
loss at epoch 2: 4715.730469
loss at epoch 3: 4411.828125
loss at epoch 4: 4408.219727
loss at epoch 5: 3480.742676
loss at epoch 6: 3162.100342
loss at epoch 7: 2884.531982
loss at epoch 8: 2408.105713
loss at epoch 9: 2086.409668
loss at epoch 10: 1728.721558
generated sentence: <START>a very clean and well <UNK> empty bathroom <END> with and a appliances and in laying 
loss at epoch 11: 1423.850342
loss at epoch 12: 1198.077515
loss at epoch 13: 1000.056396
loss at epoch 14: 826.243469
loss at epoch 15: 677.567322
loss at epoch 16: 548.780212
loss at epoch 17: 440.776978
loss at epoch 18: 356.138550
loss at epoch 19: 285.039642
loss at epoch 20: 227.286255
generated sentence: <START>a very clean and well decorated empty bathroom <END> with and a standing and <UNK> an 
loss at epoch 21: 183.